In [106]:
import tensorflow as tf
from tensorflow import keras, sparse 
from keras import layers, applications
# from keras.preprocessing import image
from keras.applications.efficientnet import EfficientNetB0, EfficientNetB5, EfficientNetB7
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.densenet import DenseNet121
# from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator


import os 
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential

# Data preprocessing 
from keras.preprocessing.image import load_img, img_to_array
 

In [117]:

# subset IN (TRAINING, VALIDATION)
"""
      validation_split=0.05,
      subset= subset,
"""
def getDataset(dataFolder, subset, imageSize = (224, 224), batchSize = 32):
    train_ds = keras.utils.image_dataset_from_directory(
      dataFolder,
      seed=123,
      image_size=imageSize,
      batch_size=batchSize)
    return train_ds


def configurePerformance(train_ds, val_ds): 
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    return train_ds, val_ds

def buildModel(dropoutRate, numClasses):
    inp = layers.Input(shape=inp_shape)
    baseModel = EfficientNetB0(weights="imagenet",
                       include_top = False) 
    baseModel.trainable = False 
    x = baseModel(inp, training=False)
    x =  layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.Dropout(dropoutRate, noise_shape=None, seed=None)(x)
    out = layers.Dense(numClasses,activation="softmax", name = "pred")(x)
    model = keras.Model(inp, out, name="B0")
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
    
    return model 


In [118]:
EPOCHS = 10

trainingFolder = 'data/x5_RGB/train/RGB/'
testingFolder = 'data/x5_RGB/test_with_labels/RGB/'
train_ds = getDataset(trainingFolder, "training")
val_ds =  getDataset(testingFolder, "validation")
model = buildModel(0.2, 7)


Found 6000 files belonging to 7 classes.
Found 1500 files belonging to 7 classes.


In [119]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)


Epoch 1/10


2022-03-05 17:27:18.851054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - ETA: 0s - loss: 0.7443 - accuracy: 0.7705

2022-03-05 17:28:02.496578: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 [==============================] - 62s 307ms/step - loss: 0.7443 - accuracy: 0.7705 - val_loss: 0.9447 - val_accuracy: 0.6440
Epoch 2/10
188/188 [==============================] - 53s 278ms/step - loss: 0.3746 - accuracy: 0.8872 - val_loss: 0.8981 - val_accuracy: 0.6633
Epoch 3/10
188/188 [==============================] - 52s 276ms/step - loss: 0.2991 - accuracy: 0.9062 - val_loss: 0.8813 - val_accuracy: 0.6693
Epoch 4/10
188/188 [==============================] - 52s 274ms/step - loss: 0.2605 - accuracy: 0.9183 - val_loss: 0.8981 - val_accuracy: 0.6707
Epoch 5/10
188/188 [==============================] - 53s 283ms/step - loss: 0.2338 - accuracy: 0.9267 - val_loss: 0.8884 - val_accuracy: 0.6867
Epoch 6/10
188/188 [==============================] - 53s 282ms/step - loss: 0.2151 - accuracy: 0.9313 - val_loss: 0.8949 - val_accuracy: 0.6867
Epoch 7/10
188/188 [==============================] - 53s 283ms/step - loss: 0.2015 - accuracy: 0.9367 - val_loss: 0.9045 - val_accuracy: 0.6

In [125]:
import pickle

# Save the trained model as a pickle string.
pickle.dump(model, open('model.pkl', 'wb'))




INFO:tensorflow:Assets written to: ram://cb986ee5-de15-4079-9011-3659a85c3883/assets


In [126]:
preds = model.predict(val_ds, verbose = 1)


2022-03-05 17:47:34.977248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 14s 246ms/step


In [130]:
def getImageRepresentation():
    import random
    pred_labels = tf.argmax(preds, axis=1)
    test_labels = np.concatenate([y for x, y in val_ds], axis=0)
    test_image_batches = []
    for images, labels in val_ds.take(-1):
        test_image_batches.append(images.numpy())

    test_images = [item for sublist in test_image_batches for item in sublist]
    plt.figure(figsize = (20,20))
    for i in range(9):
        random_int_index = random.choice(range(len(test_images)))
        plt.subplot(3,3,i+1)
        plt.imshow(test_images[random_int_index]/255.)
        if test_labels[random_int_index] == pred_labels[random_int_index]:
            color = "g"
        else:
            color = "r"
        plt.title("True Label: " + class_names[test_labels[random_int_index]] + " || " + "Predicted Label: " +
                  class_names[pred_labels[random_int_index]] + "\n" + 
                  str(np.asarray(tf.reduce_max(preds, axis = 1))[random_int_index]), c=color)
        plt.axis(False);

In [131]:
model.evaluate(val_ds)

47/47 [==============================] - 12s 228ms/step - loss: 0.9283 - accuracy: 0.7053


[0.9282761216163635, 0.7053333520889282]